In [2]:
##import packages

import requests
import json
import pandas as pd
import numpy as np
from decimal import Decimal
import itertools as it

In [3]:
##Request fixtures from FPL API

url = f"https://fantasy.premierleague.com/api/fixtures/"
response = requests.get(url)
json_data = response.json()

Fixtures_Dim = json_data

Fixtures_Dim = pd.json_normalize(Fixtures_Dim)


Fixtures_Dim_Future=Fixtures_Dim[Fixtures_Dim['finished']==False]

### Used as variable to return only the data for completed gameweeks

UpcomingWeek = Fixtures_Dim_Future['event'].min().astype(int)
PrevWeek = UpcomingWeek-1

### Variable for number of gameweeks to go back to calculate averages from

NoWeeks = 4

TeamID = 1769272

print(UpcomingWeek)
print(PrevWeek)


16
15


In [4]:
###Request Team Data from FPL API

url = f"https://fantasy.premierleague.com/api/bootstrap-static"
response = requests.get(url)
json_data = response.json()

teams_dim = json_data['teams']

Teams_Dim = pd.json_normalize(teams_dim)



In [5]:
###Duplicates of Fixtures for Home and Away, so that this can be merged with elements

Fixtures_Away1 = Fixtures_Dim_Future
Fixtures_Away1.head()

Fixtures_Home1 = Fixtures_Dim_Future
Fixtures_Home1.head()

FixturesConcat = pd.concat([Fixtures_Away1,Fixtures_Home1], keys=["Away", "Home"])

FixturesConcat.reset_index(inplace=True)

FixturesConcat['TeamID'] = FixturesConcat.apply(
    lambda row: row['team_a'] if row['level_0'] == 'Away' else row['team_h'], 
    axis=1
)

FixturesConcat['OppTeam2'] = FixturesConcat.apply(
    lambda row: row['team_a'] if row['level_0'] == 'Home' else row['team_h'], 
    axis=1
)

In [6]:
### Gameweek elements data from FPL API

player_data = []

for gw1 in range(1,PrevWeek):
    url = f"https://fantasy.premierleague.com/api/event/{gw1}/live/"
    response = requests.get(url)
    json_data = response.json()

    for gw in json_data['elements']:
        #i1 = gw['id']
        playerid=gw['id']
        minutes=gw['stats']['minutes']
        goals_scored=gw['stats']['goals_scored']
        assists=gw['stats']['assists']
        clean_sheets=gw['stats']['clean_sheets']
        goals_conceded=gw['stats']['goals_conceded']
        own_goals=gw['stats']['own_goals']
        penalties_saved=gw['stats']['penalties_saved']
        penalties_missed=gw['stats']['penalties_missed']
        yellow_cards=gw['stats']['yellow_cards']
        red_cards=gw['stats']['red_cards']
        saves=gw['stats']['saves']
        bonus=gw['stats']['bonus']
        bps=gw['stats']['bps']
        influence=gw['stats']['influence']
        creativity=gw['stats']['creativity']
        threat=gw['stats']['threat']
        ict_index=gw['stats']['ict_index']
        starts=gw['stats']['starts']
        expected_goals=gw['stats']['expected_goals']
        expected_assists=gw['stats']['expected_assists']
        expected_goal_involvements=gw['stats']['expected_goal_involvements']
        expected_goals_conceded=gw['stats']['expected_goals_conceded']
        total_points=gw['stats']['total_points']
        fixtureid=gw['explain'][0]['fixture']

        player_data.append({
            'playerid': playerid,
            'minutes': minutes,
            'goals_scored': goals_scored,
            'assists': assists,
            'clean_sheets': clean_sheets,
            'goals_conceded': goals_conceded,
            'threat': threat,
            'expected_goals': expected_goals,
            'expected_assists': expected_assists,
            'expected_goal_involvements': expected_goal_involvements,
            'expected_goals_conceded': expected_goals_conceded,
            'fixtureid': fixtureid  


            })

    GWElementsDF = pd.DataFrame(player_data)

GWElementsDF1 = pd.merge(GWElementsDF, Fixtures_Dim,left_on='fixtureid', right_on='id', how='outer')

GWElementsDF1.to_csv('FPL_Event_By_Week.csv')
GWElementsDF1.head()


,playerid,minutes_x,goals_scored,assists,clean_sheets,goals_conceded,threat,expected_goals,expected_assists,expected_goal_involvements,...,provisional_start_time,started,team_a,team_a_score,team_h,team_h_score,stats,team_h_difficulty,team_a_difficulty,pulse_id
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,...,False,True,20,0.0,1,2.0,"[{'identifier': 'goals_scored', 'a': [], 'h': ...",2,5,115828
1,2.0,5.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,...,False,True,20,0.0,1,2.0,"[{'identifier': 'goals_scored', 'a': [], 'h': ...",2,5,115828
2,3.0,90.0,0.0,0.0,1.0,0.0,8.0,0.12,0.02,0.14,...,False,True,20,0.0,1,2.0,"[{'identifier': 'goals_scored', 'a': [], 'h': ...",2,5,115828
3,4.0,90.0,1.0,1.0,1.0,0.0,46.0,0.45,0.04,0.49,...,False,True,20,0.0,1,2.0,"[{'identifier': 'goals_scored', 'a': [], 'h': ...",2,5,115828
4,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,...,False,True,20,0.0,1,2.0,"[{'identifier': 'goals_scored', 'a': [], 'h': ...",2,5,115828


In [7]:
###Standard Element Data from FPL API

url = f"https://fantasy.premierleague.com/api/bootstrap-static"
response = requests.get(url)
json_data = response.json()

elements_dim = json_data['elements']

Elements_Dim1 = pd.json_normalize(elements_dim)

Elements_Dim1.to_csv('elementsdim.csv')

Elements_Dim1 = pd.merge(GWElementsDF1, Elements_Dim1,left_on='playerid', right_on='id', how='outer',suffixes=('.gw', '.info'))

Elements_Dim1['ExcludeDef'] = Elements_Dim1.apply(
    lambda row: 'Y' if (row['element_type'] == '1' or row['element_type'] == '2') and row['minutes_x'] > 59 else 'N', 
    axis=1
)

Elements_Dim1.to_csv('elements4wk.csv')

In [8]:
###Arbitrary team weightings calculated based on average of last season's odds

TeamWeightings = {
    "Weighting": [
        1.699153895, 1.68275948, 1.639154108, 1.626396476, 1.605876872, 1.542304678,
        1.526424188, 1.50219836, 1.499125082, 1.489893392, 1.44492154, 1.424635514,
        1.365324148, 1.288771283, 1.266010528, 1.251813146, 1.18226904, 1.7, 1.7, 1.7
    ],
    "Teams.id": [
        3, 7, 16, 20, 4, 9, 8, 2, 15, 19, 5, 14, 6, 12, 18, 1, 13, 10, 11, 17
    ],
    "Teams.name": [
        "Bournemouth", "Crystal Palace", "Nott'm Forest", "Wolves", "Brentford", "Fulham",
        "Everton", "Aston Villa", "Newcastle", "West Ham", "Brighton", "Man Utd", "Chelsea",
        "Liverpool", "Spurs", "Arsenal", "Man City", "Ipswich", "Leicester", "Southampton"
    ]
}

TeamWeightings = pd.DataFrame(TeamWeightings)

Elements_Dim1['OppTeam'] = Elements_Dim1.apply(lambda row: row['team_h'] if row['team_h'] == row['team'] else row['team_a'], axis=1)

Elements_Dim2 = pd.merge(Elements_Dim1, TeamWeightings,left_on='OppTeam', right_on='Teams.id', how='outer')

Elements_Dim2['XGC_Weighted'] = Elements_Dim2['Weighting'].astype(float)*Elements_Dim2['expected_goals_conceded.gw'].astype(float)

Elements_Dim2['XGI_Weighted'] = Elements_Dim2['expected_goal_involvements.gw'].astype(float)/Elements_Dim2['Weighting'].astype(float)



In [9]:
###Filtered to only include the previous 4 weeks' data before it is grouped to return total sum for each team 

Elements_Dim2Filtered = Elements_Dim2[Elements_Dim2['event']>PrevWeek-4]

Elements_Dim2Grouped1 = Elements_Dim2Filtered.groupby(['OppTeam', 'event']).agg(
    XGI_Weighted=('XGI_Weighted', 'sum'),
    XGC_Weighted=('XGC_Weighted', 'max')
).reset_index()

##Average of the above taken for each team
Elements_Dim2Grouped2 = Elements_Dim2Grouped1.groupby(['OppTeam']).agg(
    XGI_Weighted=('XGI_Weighted', 'mean'),
    XGC_Weighted=('XGC_Weighted', 'mean')
).reset_index()



Elements_Dim2Grouped2.head()

,OppTeam,XGI_Weighted,XGC_Weighted
0,1.0,0.585817,0.872096
1,2.0,0.440244,1.952858
2,3.0,0.446497,1.121442
3,4.0,0.283542,2.510521
4,5.0,0.326200,1.589414


In [10]:
##Previous 4 Gameweeks this dataframe is returning the averages for the selected players 
Elements_Dim1 = Elements_Dim1[Elements_Dim1['event']<UpcomingWeek]

GWElementsDFLast4Weeks = Elements_Dim1[Elements_Dim1['event']>UpcomingWeek-NoWeeks-1]
GWElementsDFLast4Weeks = Elements_Dim1[['playerid','web_name','element_type','team','now_cost','expected_goals.gw','expected_assists.gw','expected_goal_involvements.gw','expected_goals_conceded.gw']]

GWElementsDFLast4Weeks['expected_goals.gw'] = GWElementsDFLast4Weeks['expected_goals.gw'].astype(float)
GWElementsDFLast4Weeks['expected_assists.gw'] = GWElementsDFLast4Weeks['expected_assists.gw'].astype(float)
GWElementsDFLast4Weeks['expected_goal_involvements.gw'] = GWElementsDFLast4Weeks['expected_goal_involvements.gw'].astype(float)
GWElementsDFLast4Weeks['expected_goals_conceded.gw'] = GWElementsDFLast4Weeks['expected_goals_conceded.gw'].astype(float)
grouped_dfLast4Weeks = GWElementsDFLast4Weeks.groupby(['playerid','web_name','element_type','now_cost','team'], as_index=False)[['expected_goals.gw','expected_assists.gw','expected_goal_involvements.gw','expected_goals_conceded.gw']].mean(numeric_only=True)

grouped_dfLast4Weeks.head()

C:\Users\james\AppData\Local\Temp\ipykernel_16288\1589820039.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GWElementsDFLast4Weeks['expected_goals.gw'] = GWElementsDFLast4Weeks['expected_goals.gw'].astype(float)
C:\Users\james\AppData\Local\Temp\ipykernel_16288\1589820039.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GWElementsDFLast4Weeks['expected_assists.gw'] = GWElementsDFLast4Weeks['expected_assists.gw'].astype(float)
C:\Users\james\AppData\Local\Temp\ipykernel_16288\1589820039.py:9: Setting

,playerid,web_name,element_type,now_cost,team,expected_goals.gw,expected_assists.gw,expected_goal_involvements.gw,expected_goals_conceded.gw
0,1.0,Fábio Vieira,3.0,54.0,1.0,0.000000,0.000000,0.000000,0.000000
1,2.0,G.Jesus,4.0,68.0,1.0,0.062143,0.008571,0.070714,0.174286
2,3.0,Gabriel,2.0,61.0,1.0,0.125714,0.035714,0.161429,0.892857
3,4.0,Havertz,4.0,79.0,1.0,0.386429,0.082143,0.468571,0.977857
4,5.0,Hein,1.0,40.0,1.0,0.000000,0.000000,0.000000,0.000000


In [11]:
### Returns combinations of historic XG stats from players multiplied by historic opposition team performance

OpportunitiesCalc = pd.merge(grouped_dfLast4Weeks, Elements_Dim2Grouped2,left_on='team', right_on='OppTeam', how='outer')

OpportunitiesCalc['AttackOpp2'] = OpportunitiesCalc['XGC_Weighted']*OpportunitiesCalc['expected_goal_involvements.gw']

OpportunitiesCalc['DefenceOpp2'] = OpportunitiesCalc['XGI_Weighted']*OpportunitiesCalc['expected_goal_involvements.gw']

OpportunitiesCalc.sort_values(by=['AttackOpp2'])


,playerid,web_name,element_type,now_cost,team,expected_goals.gw,expected_assists.gw,expected_goal_involvements.gw,expected_goals_conceded.gw,OppTeam,XGI_Weighted,XGC_Weighted,AttackOpp2,DefenceOpp2
0,1.0,Fábio Vieira,3.0,54.0,1.0,0.000000,0.000000,0.000000,0.000000,1.0,0.585817,0.872096,0.000000,0.000000
355,668.0,Cahill,1.0,39.0,5.0,0.000000,0.000000,0.000000,0.000000,5.0,0.326200,1.589414,0.000000,0.000000
354,663.0,Samuels,2.0,40.0,5.0,0.000000,0.000000,0.000000,0.000000,5.0,0.326200,1.589414,0.000000,0.000000
350,615.0,Rushworth,1.0,40.0,5.0,0.000000,0.000000,0.000000,0.000000,5.0,0.326200,1.589414,0.000000,0.000000
349,601.0,Moran,3.0,50.0,5.0,0.000000,0.000000,0.000000,0.000000,5.0,0.326200,1.589414,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635,514.0,Bowen,3.0,74.0,19.0,0.201429,0.265714,0.467143,1.780000,19.0,0.405398,3.456553,1.614704,0.189379
551,328.0,M.Salah,3.0,134.0,12.0,0.732857,0.208571,0.941429,0.954286,12.0,0.666194,1.799984,1.694556,0.627174
382,306.0,Vardy,4.0,56.0,11.0,0.297143,0.081429,0.378571,1.831429,11.0,0.229020,4.675000,1.769821,0.086700
98,27.0,Archer,4.0,50.0,17.0,0.296429,0.010714,0.307143,1.512143,17.0,0.241176,5.814000,1.785729,0.074076


In [12]:
##### Returns Output 1 FixturesElementsAll
FixturesElementsAll1 = pd.merge(FixturesConcat, Elements_Dim2Grouped2,left_on='OppTeam2', right_on='OppTeam', how='outer')

FixturesElementsAll2 = pd.merge(grouped_dfLast4Weeks, FixturesElementsAll1,left_on='team', right_on='TeamID', how='outer')

FixturesElementsAll2['AttackOpp_pergame'] = FixturesElementsAll2['XGC_Weighted']*FixturesElementsAll2['expected_goal_involvements.gw']

FixturesElementsAll2['DefenceOpp_pergame'] = FixturesElementsAll2['XGI_Weighted']*FixturesElementsAll2['expected_goals_conceded.gw']

FixturesElementsAll2.to_csv('FixturesElementsAll.csv')

In [13]:
##### Returns Output 2 FixturesElementsAllGrouped

FixturesElementsAllGrouped = FixturesElementsAll2[FixturesElementsAll2['event']>UpcomingWeek-NoWeeks-1]
FixturesElementsAllGrouped = FixturesElementsAllGrouped[['playerid','web_name','element_type','team','now_cost','expected_goals.gw','expected_assists.gw','expected_goal_involvements.gw','expected_goals_conceded.gw','XGI_Weighted','XGC_Weighted','AttackOpp_pergame','DefenceOpp_pergame']]

FixturesElementsAllGrouped = FixturesElementsAllGrouped.groupby(['playerid','web_name','element_type','now_cost','team'], as_index=False)[['expected_goals.gw','expected_assists.gw','expected_goal_involvements.gw','expected_goals_conceded.gw','XGI_Weighted','XGC_Weighted','AttackOpp_pergame','DefenceOpp_pergame']].mean(numeric_only=True)

FixturesElementsAllGrouped.to_csv('FixturesElementsAllGrouped.csv')

In [14]:
url = f"https://fantasy.premierleague.com/api/entry/{TeamID}/event/{PrevWeek}/picks/"
response = requests.get(url)
json_data = response.json()

TeamSelection = json_data['picks']

TeamSelection = pd.json_normalize(TeamSelection)

TeamSelection.head()

,element,position,multiplier,is_captain,is_vice_captain,element_type
0,15,1,1,False,False,1
1,594,2,1,False,False,2
2,422,3,1,False,False,2
3,338,4,1,False,False,2
4,573,5,1,False,False,2


In [15]:
FixturesElementsAllGroupedwithTS = pd.merge(FixturesElementsAllGrouped, TeamSelection,left_on='playerid', right_on='element', how='outer')

FixturesElementsAllGroupedwithTS['IsInCurrentTeam'] = np.where(
    FixturesElementsAllGroupedwithTS['is_captain'].notnull(),
    'Y',
    'N'
)

In [16]:
FixturesElementsAllGroupedwithTS.head()

,playerid,web_name,element_type_x,now_cost,team,expected_goals.gw,expected_assists.gw,expected_goal_involvements.gw,expected_goals_conceded.gw,XGI_Weighted,XGC_Weighted,AttackOpp_pergame,DefenceOpp_pergame,element,position,multiplier,is_captain,is_vice_captain,element_type_y,IsInCurrentTeam
0,1.0,Fábio Vieira,3.0,54.0,1.0,0.000000,0.000000,0.000000,0.000000,0.32039,2.461602,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,N
1,2.0,G.Jesus,4.0,68.0,1.0,0.062143,0.008571,0.070714,0.174286,0.32039,2.461602,0.174070,0.055839,NaN,NaN,NaN,NaN,NaN,NaN,N
2,3.0,Gabriel,2.0,61.0,1.0,0.125714,0.035714,0.161429,0.892857,0.32039,2.461602,0.397373,0.286063,NaN,NaN,NaN,NaN,NaN,NaN,N
3,4.0,Havertz,4.0,79.0,1.0,0.386429,0.082143,0.468571,0.977857,0.32039,2.461602,1.153436,0.313296,NaN,NaN,NaN,NaN,NaN,NaN,N
4,5.0,Hein,1.0,40.0,1.0,0.000000,0.000000,0.000000,0.000000,0.32039,2.461602,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,N


In [18]:
FixturesElementsAllGroupedwithTS.to_csv('FixturesElementsAllGroupedwithTS.csv')

In [19]:
##Import the packages to carry out the optimisation problem

import requests
from sklearn.preprocessing import MinMaxScaler
import warnings
from pulp import *
Small_Elements_df = FixturesElementsAllGroupedwithTS

In [21]:
### Enter amount to spend on Midfield/Attack - Currently hardcoded but can be derived from API

AttSpend = 680



In [22]:
##currently just set the 

Small_Elements_df.head()
Small_Elements_df['element_type']=Small_Elements_df['element_type_x']

Attack_Elements_df = Small_Elements_df.dropna(subset=['AttackOpp_pergame'])
Attack_Elements_df['AttackOpp_pergame'].astype(float)
Def_Elements_df=Small_Elements_df.dropna(subset=['DefenceOpp_pergame'])
Def_Elements_df['DefenceOpp_pergame'].astype(float)
Attack_Elements_df=Attack_Elements_df[Attack_Elements_df['element_type']!=1]
Attack_Elements_df=Attack_Elements_df[Attack_Elements_df['element_type']!=2]
Def_Elements_df=Small_Elements_df[Small_Elements_df['element_type']!=3]
Def_Elements_df=Def_Elements_df[Def_Elements_df['element_type']!=4]


In [23]:
Def_Elements_df.to_csv('test123.csv')

In [24]:
## Attack Variables
Players = list(Attack_Elements_df['web_name'])
Cost = dict(zip(Players, Attack_Elements_df['now_cost']))
Score = dict(zip(Players, Attack_Elements_df['AttackOpp_pergame'].astype(float)))
#Defence = dict(zip(Players, Small_Elements_df['AttackOpp_pergame'].astype(int)))
positions = dict(zip(Players, Attack_Elements_df['element_type']))
CurrentSelection = dict(zip(Players, Attack_Elements_df['IsInCurrentTeam']))

In [25]:
player_vars = LpVariable.dicts("Player", Players, lowBound=0, upBound=1, cat='Integer')

In [26]:
##Attack Constraints and problem solve

total_score = LpProblem("Fantasy_Points_Problem", LpMaximize)
total_score += lpSum([Score[i] * player_vars[i] for i in player_vars])
total_score += lpSum([Cost[i] * player_vars[i] for i in player_vars]) <= 680



In [27]:
MF = [p for p in positions.keys() if positions[p] == 3]
ST = [p for p in positions.keys() if positions[p] == 4]
CS = [p for p in positions.keys() if CurrentSelection[p] == 'N']
total_score += lpSum([player_vars[i] for i in MF]) == 5
total_score += lpSum([player_vars[i] for i in ST]) == 3
total_score += lpSum([player_vars[i] for i in CS]) == 2


In [28]:

total_score.solve()

for v in total_score.variables():
    if v.varValue > 0:
        print(v.name)

Player_Evanilson
Player_Isak
Player_Johnson
Player_M.Salah
Player_Maddison
Player_Saka
Player_Semenyo
Player_Watkins


In [27]:
## Defence Variables
DefPlayers = list(Def_Elements_df['web_name'])
DefCost = dict(zip(DefPlayers, Def_Elements_df['now_cost']))
DefScore = dict(zip(DefPlayers, Def_Elements_df['DefenceOpp_pergame'].astype(float)))
Defpositions = dict(zip(DefPlayers, Def_Elements_df['element_type']))
DefCurrentSelection = dict(zip(DefPlayers, Def_Elements_df['IsInCurrentTeam']))

player_vars_def = LpVariable.dicts("DefPlayers", DefPlayers, lowBound=0, upBound=1, cat='Integer')

##Defence Constraints and problem solve

total_score_def = LpProblem("Fantasy_Points_Problem_Defence", LpMaximize)
total_score_def += lpSum([DefScore[i] * player_vars_def[i] for i in player_vars_def])
total_score_def += lpSum([DefCost[i] * player_vars_def[i] for i in player_vars_def]) <= 320





In [ ]:
##Defence Constraints and problem solve

GK = [p for p in positions.keys() if positions[p] == 1]
DF = [p for p in positions.keys() if positions[p] == 2]
CS = [p for p in positions.keys() if CurrentSelection[p] == 'N']


total_score_def += lpSum([player_vars[i] for i in GK]) == 2 
total_score_def += lpSum([player_vars[i] for i in DF]) == 5
total_score_def += lpSum([player_vars[i] for i in CS]) == 2

In [34]:

total_score_def.solve()

for v in total_score_def.variables():
    if v.varValue > 0:
        print(v.name)

DefPlayers_Davis
DefPlayers_Faes
DefPlayers_Harwood_Bellis
DefPlayers_Hermansen
DefPlayers_Justin
DefPlayers_Muric
DefPlayers_O'Shea
DefPlayers_Walker_Peters


In [30]:
Def_Elements_dfna1 = Def_Elements_df.dropna()

In [31]:
Def_Elements_dfna2 = Def_Elements_dfna1.isnull()
Def_Elements_dfna2.head()

,playerid,web_name,element_type_x,now_cost,team,expected_goals.gw,expected_assists.gw,expected_goal_involvements.gw,expected_goals_conceded.gw,XGI_Weighted,...,AttackOpp_pergame,DefenceOpp_pergame,element,position,multiplier,is_captain,is_vice_captain,element_type_y,IsInCurrentTeam,element_type
14,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
337,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
354,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
421,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
444,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [43]:
def AttackOptimisationproblem(Source,AttSpend,FreeTransfers):
    Attack_Elements_df = Source.dropna(subset=['AttackOpp_pergame'])
    Attack_Elements_df['AttackOpp_pergame'].astype(float)
    Attack_Elements_df=Attack_Elements_df[Attack_Elements_df['element_type']!=1]
    Attack_Elements_df=Attack_Elements_df[Attack_Elements_df['element_type']!=2]
    ## Attack Variables
    Players = list(Attack_Elements_df['web_name'])
    Cost = dict(zip(Players, Attack_Elements_df['now_cost']))
    Score = dict(zip(Players, Attack_Elements_df['AttackOpp_pergame'].astype(float)))
    positions = dict(zip(Players, Attack_Elements_df['element_type']))
    CurrentSelection = dict(zip(Players, Attack_Elements_df['IsInCurrentTeam']))
    
    player_vars = LpVariable.dicts("Player", Players, lowBound=0, upBound=1, cat='Integer')

    ##Attack Constraints and problem solve

    total_score = LpProblem("Fantasy_Points_Problem", LpMaximize)
    total_score += lpSum([Score[i] * player_vars[i] for i in player_vars])
    total_score += lpSum([Cost[i] * player_vars[i] for i in player_vars]) <= AttSpend

    MF = [p for p in positions.keys() if positions[p] == 3]
    ST = [p for p in positions.keys() if positions[p] == 4]
    CS = [p for p in positions.keys() if CurrentSelection[p] == 'N']
    total_score += lpSum([player_vars[i] for i in MF]) == 5
    total_score += lpSum([player_vars[i] for i in ST]) == 3
    total_score += lpSum([player_vars[i] for i in CS]) == FreeTransfers

    total_score.solve()

    for v in total_score.variables():
        if v.varValue > 0:
            print(v.name)


In [39]:
def DefenceOptimisationproblem(Source,DefSpend,FreeTransfers):
    Def_Elements_df=Source.dropna(subset=['DefenceOpp_pergame'])
    Def_Elements_df = Def_Elements_df[Def_Elements_df['expected_goal_involvements.gw'].astype(float)>0.1]
    Def_Elements_df['DefenceOpp_pergame'].astype(float)
    Def_Elements_df=Def_Elements_df[Def_Elements_df['element_type_x']!=3]
    Def_Elements_df=Def_Elements_df[Def_Elements_df['element_type_x']!=4]
    ## Defence Variables
    DefPlayers = list(Def_Elements_df['web_name'])
    DefCost = dict(zip(DefPlayers, Def_Elements_df['now_cost']))
    DefScore = dict(zip(DefPlayers, Def_Elements_df['DefenceOpp_pergame'].astype(float)))
    Defpositions = dict(zip(DefPlayers, Def_Elements_df['element_type_x']))
    DefCurrentSelection = dict(zip(DefPlayers, Def_Elements_df['IsInCurrentTeam']))

    player_vars_def = LpVariable.dicts("DefPlayers", DefPlayers, lowBound=0, upBound=1, cat='Integer')

##Defence Constraints and problem solve

    total_score_def = LpProblem("Fantasy_Points_Problem_Defence", LpMinimize)
    total_score_def += lpSum([DefScore[i] * player_vars_def[i] for i in player_vars_def])
    total_score_def += lpSum([DefCost[i] * player_vars_def[i] for i in player_vars_def]) <= DefSpend

    GK = [p for p in Defpositions.keys() if Defpositions[p] == 1]
    DF = [p for p in Defpositions.keys() if Defpositions[p] == 2]
    CS = [p for p in DefCurrentSelection.keys() if DefCurrentSelection[p] == 'N']
    total_score_def += lpSum([player_vars_def[i] for i in GK]) == 2
    total_score_def += lpSum([player_vars_def[i] for i in DF]) == 5
    total_score_def += lpSum([player_vars_def[i] for i in CS]) == FreeTransfers

    total_score_def.solve()

    for v in total_score_def.variables():
        if v.varValue > 0:
            print(v.name)

In [44]:
AttackOptimisationproblem(FixturesElementsAllGroupedwithTS,680,4)

Player_Evanilson
Player_Johnson
Player_M.Salah
Player_Palmer
Player_Raúl
Player_Saka
Player_Semenyo
Player_Watkins


RecursionError: maximum recursion depth exceeded in comparison

In [40]:
DefenceOptimisationproblem(FixturesElementsAllGroupedwithTS,320,2)

DefPlayers_Davis
DefPlayers_Faes
DefPlayers_Harwood_Bellis
DefPlayers_Hermansen
DefPlayers_Justin
DefPlayers_Muric
DefPlayers_O'Shea
DefPlayers_Walker_Peters
